In [1]:
%cd /content
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

%cd /content/gaussian-splatting
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn

!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
!unzip tandt_db.zip

/content
Cloning into 'gaussian-splatting'...
remote: Enumerating objects: 603, done.
remote: Total 603 (delta 0), reused 0 (delta 0), pack-reused 603 (from 1)
Receiving objects: 100% (603/603), 2.09 MiB | 18.79 MiB/s, done.
Resolving deltas: 100% (347/347), done.
Submodule 'SIBR_viewers' (https://gitlab.inria.fr/sibr/sibr_core) registered for path 'SIBR_viewers'
Submodule 'submodules/diff-gaussian-rasterization' (https://github.com/graphdeco-inria/diff-gaussian-rasterization) registered for path 'submodules/diff-gaussian-rasterization'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/gaussian-splatting/SIBR_viewers'...
remote: Enumerating objects: 3270, done.        
remote: Counting objects: 100% (299/299), done.        
remote: Compressing objects: 100% (164/164), done.        
remote: Total 3270 (delta 157), reused 259 (delta 135), pack-reused 2971 (from 1)        
Receiving objects:

In [39]:
%cd /content/gaussian-splatting
!python train.py -s /content/gaussian-splatting/tandt/train

/content/gaussian-splatting
2024-12-19 15:10:58.283567: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-19 15:10:58.303668: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-19 15:10:58.309584: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-19 15:10:58.324484: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-19 15:10:59.33836

In [13]:
!pip install PlyData

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.5/299.5 kB 6.8 MB/s eta 0:00:00


In [122]:
%cd /content/gaussian-splatting
from plyfile import PlyData, PlyElement
import numpy as np
#from utils.read_write_model import read_model, write_points3D_binary, read_points3D_binary
#from scene.dataset_readers import fetchPly

/content/gaussian-splatting


In [176]:
%cd /content
path = "/content/gaussian-splatting/output/46854a7c-4/point_cloud/iteration_30000/point_cloud.ply"
pc = PlyData.read(path)
vertices = pc['vertex']
vertex_data = vertices.data  # Access the vertex data
vertex_dtype = vertex_data.dtype
vertex_array = np.array([list(row) for row in vertices])

/content


In [40]:
%cd /content
!pip install faiss-gpu

/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.2 MB/s eta 0:00:00


In [181]:
import faiss

combined = np.ascontiguousarray(vertex_array[:,0:6])
kmeans = faiss.Kmeans(d=combined.shape[1], k=50, niter=1000, verbose=True)
kmeans.train(combined)
D, I = kmeans.index.search(combined, 1)


In [180]:
import os
%cd /content

output_dir = "clusters"
os.makedirs(output_dir, exist_ok=True)

for cluster_idx in range(kmeans.k):
    cluster_points = vertex_array[I.flatten() == cluster_idx]
    cluster_vertex_data = np.core.records.fromarrays(cluster_points.T, dtype=vertex_dtype)
    cluster_vertex_element = PlyElement.describe(cluster_vertex_data, 'vertex')
    output_path = os.path.join(output_dir, f"cluster_{cluster_idx}.ply")
    PlyData([cluster_vertex_element], text=False, byte_order= '<').write(output_path)



/content


In [164]:
cl1 = PlyData.read("/content/clusters1/cluster_1.ply")
cl1

PlyData((PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('nx', 'float'), PlyProperty('ny', 'float'), PlyProperty('nz', 'float'), PlyProperty('f_dc_0', 'float'), PlyProperty('f_dc_1', 'float'), PlyProperty('f_dc_2', 'float'), PlyProperty('f_rest_0', 'float'), PlyProperty('f_rest_1', 'float'), PlyProperty('f_rest_2', 'float'), PlyProperty('f_rest_3', 'float'), PlyProperty('f_rest_4', 'float'), PlyProperty('f_rest_5', 'float'), PlyProperty('f_rest_6', 'float'), PlyProperty('f_rest_7', 'float'), PlyProperty('f_rest_8', 'float'), PlyProperty('f_rest_9', 'float'), PlyProperty('f_rest_10', 'float'), PlyProperty('f_rest_11', 'float'), PlyProperty('f_rest_12', 'float'), PlyProperty('f_rest_13', 'float'), PlyProperty('f_rest_14', 'float'), PlyProperty('f_rest_15', 'float'), PlyProperty('f_rest_16', 'float'), PlyProperty('f_rest_17', 'float'), PlyProperty('f_rest_18', 'float'), PlyProperty('f_rest_19', 'float'), PlyProperty('f_res

In [182]:
!pip install torchvision

In [185]:
import torch
import torchvision
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_V2_Weights
from PIL import Image
import torchvision.transforms as T

# Load the pre-trained model with default weights
weights = MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT
model = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(weights=weights)
model.eval()  # Set the model to evaluation mode

# Load and preprocess the image
image_path = "/content/gaussian-splatting/tandt/train/images/00001.jpg"
image = Image.open(image_path).convert("RGB")

preprocess = weights.transforms()
input_image = preprocess(image)  # Preprocessed image as a PyTorch tensor

# Perform inference
with torch.no_grad():
    predictions = model([input_image])  # The model expects a list of images

# Extract predictions for the first image
output = predictions[0]
boxes = output['boxes'].numpy()       # Bounding boxes
labels = output['labels'].numpy()     # Class labels
scores = output['scores'].numpy()     # Confidence scores
masks = output['masks'].numpy()       # Segmentation masks (binary masks)

In [209]:
from

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]],


       [[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]],


       [[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]],


       ...,


       [[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]